# Text Analytics Exploration

## Import Data

In [1]:
url = "http://hansardpublic.parliament.sa.gov.au/Pages/HansardResult.aspx#/docid/HANSARD-10-25756"
text = "When I tried to get a copy of the commissioner's report after being tabled, why was I basically told that there was a very limited— The PRESIDENT: This is a matter of personal explanation in a supplementary. Just please, the Hon. Mr Wortley, ask your supplementary. The Hon. R.P. WORTLEY: Why weren't all members of parliament given a copy of the royal commission's report? The Hon. D.W. Ridgway: But you told us before you never read reports. The Hon. R.I. LUCAS (Treasurer) (15:26): Mr President, I won't go down that particular path, as delicious as that interjection might have been in relation to the Hon. Mr Wortley saying he couldn't trust himself to read his own reports. I don't know why the Hon. Mr Wortley was unable to get a copy of the royal commission report. It was certainly publicly available. If it pleases the member, I will see whether there is not a spare copy somewhere. If we do find a spare copy and give it to him, I will be asking questions afterwards of the honourable member just to make sure he did read it. The Hon. D.W. Ridgway: Do you want it delivered to Scuzzi or something more convenient for you? The PRESIDENT: Are you finished, the Hon. Mr Ridgway? The Hon. R.P. WORTLEY: You just worry about our trade exports, mate, for the state. The PRESIDENT: The Hon. Mr Wortley, I am waiting patiently here to give you the call for your question. Have you finished your private conversation with the Hon. Mr Ridgway? Yes? The Hon. Mr Wortley."
title = "Murray-Darling Basin Royal Commission"

In [2]:
# Import data from spreadsheet
import pandas as pd
import numpy as np
import os

data = pd.read_excel ("..\\data\\HANSARDfullDataset.xlsx", sheet_name="text")
df = pd.DataFrame(data, columns= ['hansardID','text'])
df = df.astype({"hansardID":'str', "text":'str'}) 

#df.dtypes 
df.head(2)

,hansardID,text
0,HANSARD-11-27883.xml,Statutes Amendment (Vehicle Inspections and So...
1,HANSARD-11-27883.xml,Second Reading


In [ ]:
hansardFilesInfo = pd.read_excel ("..\\data\\HANSARDfullDataset.xlsx", sheet_name="HANSARDFilesInfo")
hansardFilesInfo = pd.DataFrame(hansardFilesInfo, columns= ['FileName','URL'])
hansardFilesInfo = hansardFilesInfo.astype({"FileName":'str', "URL":'str'}) 
hansardFilesInfo.head(2)

In [ ]:
header = pd.read_excel ("..\\data\\HANSARDfullDataset.xlsx", sheet_name="header")
header = pd.DataFrame(header)
header.head(2)

In [ ]:
bill = pd.read_excel ("..\\data\\HANSARDfullDataset.xlsx", sheet_name="bill")
bill = pd.DataFrame(bill, columns= ['question','bname'])
bill.head(2)

In [3]:
# Group text into one document
grouped_text = df.groupby('hansardID')['text'].agg(lambda col: '. '.join(col))
grouped_text_df = pd.DataFrame(grouped_text, columns= ['text'])
grouped_text_df.head(5)

,text
hansardID,
HANSARD-10-21289.xml,Bills. Children and Young People (Safety) Bill...
HANSARD-10-21290.xml,Sentencing Bill. Assent. His Excellency the Go...
HANSARD-10-21291.xml,Statutes Amendment (Possession of Firearms and...
HANSARD-10-21292.xml,Public Interest Disclosure Bill. Conference. T...
HANSARD-10-21310.xml,Bills. Statutes Amendment (Heavy Vehicles Regi...


In [4]:
grouped_text.iloc[4].replace("..",".")

"Bills. Statutes Amendment (Heavy Vehicles Registration Fees) Bill. Second Reading. Adjourned debate on second reading. (Continued from 1 June 2017.). The Hon. J.M.A. LENSINK (16:35):  I rise to indicate opposition support for this bill, which is part of the harmonisation of national laws which relate to heavy vehicles. In particular, this bill amends the Highways Act and the Motor Vehicles Act so that South Australia can meet its obligations under the Heavy Vehicle National Law (South Australia) Act 2013, which contains the national law as a schedule. This bill provides for the creation of a national heavy vehicle regulator. For the benefit of readers, heavy vehicles are defined as trucks with a gross vehicle mass of 4.5 tonnes or more. The section relating to registration of the national law has not commenced yet, so heavy vehicle registrations remain under state legislation; however, those jurisdictions which are participants, which I understand includes all states and territories e

## Basic Text Analytics

### Word Count

In [5]:
import re

# Count number of words in sentence using regex
grouped_text_df['WordCount'] = grouped_text_df.apply(lambda x: len(re.findall(r'\w+', x.text)), axis=1)
grouped_text_df.head(2)

,text,WordCount
hansardID,,
HANSARD-10-21289.xml,Bills. Children and Young People (Safety) Bill...,16
HANSARD-10-21290.xml,Sentencing Bill. Assent. His Excellency the Go...,11


### Length of Sentences

In [6]:
from statistics import median

all_text = df['text'].agg(lambda col: ''.join(col))
all_text = ". ".join(all_text)

# Average number of words in a sentence
parts = [len(l.split()) for l in re.split(r'[?!.:] ', ' '.join(all_text)) if l.strip()]
print("Average = ", sum(parts)/len(parts))

# Median number of words in a sentence
print("Median = ", median(parts))
print("Min = ", min(parts))
print("Max = ", max(parts))
print("Q1 = ", np.percentile(parts,25))
print("Q3 = ", np.percentile(parts,75))

Average =  60.15269174579986
Median =  31.0
Min =  1
Max =  5479
Q1 =  6.0
Q3 =  95.0


## Document Summarisation

Articles and libraries to look into further: 
* https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
* https://stackabuse.com/text-summarization-with-nltk-in-python/
* https://github.com/alanbuxton/PyTeaserPython3
* https://github.com/abisee/pointer-generator
* https://github.com/DerwenAI/pytextrank
* https://github.com/tensorflow/models/tree/master/research/textsum
* https://radimrehurek.com/gensim/models/lsimodel.html
* https://towardsdatascience.com/text-summarization-in-python-76c0a41f0dc4 (additional links to articles at the end)

In [ ]:
# Topic Model summarisation
from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
model = LsiModel(common_corpus, id2word=common_dictionary)
vectorized_corpus = model[common_corpus]
#print(vectorized_corpus)
#model.print_topics(1)

In [ ]:
# The feature base model extracts the features of the sentence, then evaluate its importance
# Feature base text summarization by TextTeaser
#from pyteaser import SummarizeUrl
#url = 'http://www.huffingtonpost.com/2013/11/22/twitter-forward-secrecy_n_4326599.html'
#summaries = SummarizeUrl(url)
#print summaries

In [ ]:
# TextTeasor - automatic summarization algorithm that combines the power of natural language processing and machine learning
#from textteaser import TextTeaser
#tt = TextTeaser()
#tt.summarize(title, text)

### Truncated Sentences

In [7]:
def smart_truncate(content, length=200, suffix='...'):
    if len(content) <= length:
        return content
    else:
        return ' '.join(content[:length+1].split(' ')[0:-1]) + suffix

smart_truncate(text)

"When I tried to get a copy of the commissioner's report after being tabled, why was I basically told that there was a very limited— The PRESIDENT: This is a matter of personal explanation in a..."

In [8]:
grouped_text_df['Truncated'] = grouped_text_df.apply(lambda x: smart_truncate(x.text), axis=1)
grouped_text_df.head(5)

,text,WordCount,Truncated
hansardID,,,
HANSARD-10-21289.xml,Bills. Children and Young People (Safety) Bill...,16,Bills. Children and Young People (Safety) Bill...
HANSARD-10-21290.xml,Sentencing Bill. Assent. His Excellency the Go...,11,Sentencing Bill. Assent. His Excellency the Go...
HANSARD-10-21291.xml,Statutes Amendment (Possession of Firearms and...,18,Statutes Amendment (Possession of Firearms and...
HANSARD-10-21292.xml,Public Interest Disclosure Bill. Conference. T...,69,Public Interest Disclosure Bill. Conference. T...
HANSARD-10-21310.xml,Bills. Statutes Amendment (Heavy Vehicles Regi...,327,Bills. Statutes Amendment (Heavy Vehicles Regi...


### TextRank Summary

In [9]:
from gensim.summarization.summarizer import summarize

#  TextRank summarization with default parameters
grouped_text_df['TextRank'] = grouped_text_df.apply(lambda x: summarize(x.text), axis=1)

#  TextRank summarization with no more than 50 words for the summary
grouped_text_df['TextRank50'] = grouped_text_df.apply(lambda x: summarize(x.text, word_count = 50), axis=1)
grouped_text_df.head(5)

,text,WordCount,Truncated,TextRank,TextRank50
hansardID,,,,,
HANSARD-10-21289.xml,Bills. Children and Young People (Safety) Bill...,16,Bills. Children and Young People (Safety) Bill...,,Bills.\nChildren and Young People (Safety) Bil...
HANSARD-10-21290.xml,Sentencing Bill. Assent. His Excellency the Go...,11,Sentencing Bill. Assent. His Excellency the Go...,,
HANSARD-10-21291.xml,Statutes Amendment (Possession of Firearms and...,18,Statutes Amendment (Possession of Firearms and...,,
HANSARD-10-21292.xml,Public Interest Disclosure Bill. Conference. T...,69,Public Interest Disclosure Bill. Conference. T...,"MALINAUSKAS (Minister for Police, Minister for...","MALINAUSKAS (Minister for Police, Minister for..."
HANSARD-10-21310.xml,Bills. Statutes Amendment (Heavy Vehicles Regi...,327,Bills. Statutes Amendment (Heavy Vehicles Regi...,Adjourned debate on second reading..\nThis bil...,Adjourned debate on second reading..\nAmendmen...


In [10]:
print(summarize(text)) #  TextRank summarization

The Hon. R.P. WORTLEY: Why weren't all members of parliament given a copy of the royal commission's report?
The Hon. D.W. Ridgway: But you told us before you never read reports.
I don't know why the Hon. Mr Wortley was unable to get a copy of the royal commission report.


In [11]:
print(summarize(text, word_count = 50)) #  TextRank summarization - no more than 50 words for summary

The Hon. R.P. WORTLEY: Why weren't all members of parliament given a copy of the royal commission's report?
The Hon. D.W. Ridgway: But you told us before you never read reports.
I don't know why the Hon. Mr Wortley was unable to get a copy of the royal commission report.


In [12]:
print(summarize(text, ratio = 0.2)) #  TextRank summarization - use no more than 20% of original text for summary

The Hon. R.P. WORTLEY: Why weren't all members of parliament given a copy of the royal commission's report?
The Hon. D.W. Ridgway: But you told us before you never read reports.
I don't know why the Hon. Mr Wortley was unable to get a copy of the royal commission report.


In [13]:
summarize(grouped_text.iloc[4], word_count = 50)

"Adjourned debate on second reading..\nAmendments to the Motor Vehicles Act clarify that deductions from concessional registration charges for people living in remote areas and from primary producers will be taken from the roads component rather than the regulatory component of the fees provided to the regulator's fund.."

In [14]:
summarize(grouped_text.iloc[2], word_count = 50)

''

In [ ]:
def generate_textrank_summary(original_text):
    sentences = []    
    text = re.split(r'[?!.:] ', original_text)
    
    for sentence in text:
        processed = sentence.replace("[^a-zA-Z]", " ")
        word_count = len(re.findall(r'\w+', processed)) 
        if word_count > 3: # Include sentences with more than three words
            sentences.append(processed)
    
    text = '. '.join(sentences)
    
    if len(text) == 0 or "." not in text:
        text = original__text
    
    summary = summarize(text)             
    return summary.replace("\n"," ").replace("..",".")
    
#  TextRank summarization with no more than 50 words for the summary
grouped_text_df['TextRankProcessed'] = grouped_text_df.apply(lambda x: generate_textrank_summary(x.text), axis=1)
grouped_text_df.head(5)

In [ ]:
generate_textrank_summary(grouped_text.iloc[4])

In [ ]:
generate_textrank_summary(grouped_text.iloc[2])

In [ ]:
generate_textrank_summary(text)

In [15]:
# https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70
# Approach uses TextRank algorithm
# TextRank does not rely on any previous training data and can work with any arbitrary piece of text. 
# TextRank is a general purpose graph-based ranking algorithm for NLP

# Import all necessary libraries
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import re 

# Generate clean sentences
def read_article(file_name, output=True):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = re.split(r'[?!.:] ', filedata[0])
    return create_clean_sentences(article, output)

def read_text(text, output=True):
    article = re.split(r'[?!.:] ', text)
    return create_clean_sentences(article, output)

def create_clean_sentences(article, output=True):
    sentences = []
    
    for sentence in article:
        processed = sentence.replace("[^a-zA-Z]", " ")
        word_count = len(re.findall(r'\w+', processed)) 
        if word_count > 4: # Include sentences with more than five words
            sentences.append(processed.split(" "))
        if output: 
            print(sentence, ": words = ", word_count)
    
    #sentences.pop() 
    
    return sentences   

# Similarity matrix
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

# Generate Summary Method
def generate_summary(file_name, text, top_n=5, output=True):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text and split it
    sentences = ""
    if file_name is not None:
        sentences = read_article(file_name, output)
    else:
        sentences = read_text(text, output)

    # Step 2 - Generate Similarly Matrix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    try: 
        scores = nx.pagerank(sentence_similarity_graph, max_iter=100)
    except nx.NetworkXError:
        return ""
    except nx.PowerIterationFailedConvergence:
        return ""
    
    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    if output:
        print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    if len(ranked_sentence) < top_n:
        top_n = len(ranked_sentence)
        
    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - output the summary text        
    return '. '.join(summarize_text).replace("..",".")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\noaka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# let's begin
generate_summary(os.getcwd() + "\\test.txt", None, 2, False)

'This program also included developer-focused AI school that provided a bunch of assets to help build AI skills. Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services'

In [17]:
generate_summary(None, grouped_text.iloc[4], 2, False)

'This bill provides for the creation of a national heavy vehicle regulator. In particular, this bill amends the Highways Act and the Motor Vehicles Act so that South Australia can meet its obligations under the Heavy Vehicle National Law (South Australia) Act 2013, which contains the national law as a schedule'

In [18]:
generate_summary(None, grouped_text.iloc[2], 2, True)

Statutes Amendment (Possession of Firearms and Prohibited Weapons) Bill : words =  9
Assent : words =  1
His Excellency the Governor assented to the bill. : words =  8
Indexes of top ranked_sentence order are  [(0.5, ['Statutes', 'Amendment', '(Possession', 'of', 'Firearms', 'and', 'Prohibited', 'Weapons)', 'Bill']), (0.5, ['His', 'Excellency', 'the', 'Governor', 'assented', 'to', 'the', 'bill.'])]


'Statutes Amendment (Possession of Firearms and Prohibited Weapons) Bill. His Excellency the Governor assented to the bill.'

In [19]:
generate_summary(None, text, 2, False)

"Mr Wortley was unable to get a copy of the royal commission report. Mr Wortley saying he couldn't trust himself to read his own reports"

In [ ]:
# Iterate over all text in data frame and add summary
grouped_text_df['TextRankCustom'] = grouped_text_df.apply(lambda x: generate_summary(None, x.text, 2, False), axis=1)

c:\users\noaka\appdata\local\programs\python\python37-32\lib\site-packages\nltk\cluster\util.py:133: RuntimeWarning: invalid value encountered in true_divide
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))


In [ ]:
grouped_text_df.head(5)

### Excel Output of Document Summaries

In [ ]:
grouped_text_df.to_excel('.\\DocumentSummary.xlsx', sheet_name='TextRank', index=True)